# Test de metricas

In [ ]:
import glob
import os
import natsort
import numpy as np
from torchmetrics import Dice, JaccardIndex, Precision, Recall, Specificity, Accuracy
import re
from scipy.io import loadmat
import skvideo.io as sio
import random
import torchio as tio
import skimage.morphology as skm
from skimage import measure
import cv2
import matplotlib.pyplot as plt
import torch
from miseval import evaluate
import pandas as pd

ground_truth_path = '/data/DATA/private_data/Breast studyrc2'
videos_path = '/data/DATA/private_data/Breast studyrc2/'


# --- regex y helpers ---
mat_idx_re_sweep = re.compile(r'_sweep(\d+)\.mat$', re.IGNORECASE)
mat_idx_re_dash  = re.compile(r'-(\d+)\.mat$', re.IGNORECASE)
npy_idx_re       = re.compile(r'-(\d+)_.*masken\.npy$', re.IGNORECASE)
video_idx_re     = re.compile(r'-(\d+)\.mp4$', re.IGNORECASE)  # videos: ...-#id.mp4

def get_mat_idx(p):
    base = os.path.basename(p)
    m = mat_idx_re_sweep.search(base)
    if m: return int(m.group(1))
    m = mat_idx_re_dash.search(base)
    if m: return int(m.group(1))
    return None

def get_npy_idx(p):
    m = npy_idx_re.search(os.path.basename(p))
    return int(m.group(1)) if m else None

def get_video_idx(p):
    m = video_idx_re.search(os.path.basename(p))
    return int(m.group(1)) if m else None

#obtain subfolder ids in /data/GitHub/Breast-AI-model/src/predictions_rc2/
def get_subfolder_ids(path):
    subfolders = glob.glob(os.path.join(path, '*/'))
    subfolder_ids = [os.path.basename(os.path.normpath(folder)) for folder in subfolders]
    return natsort.natsorted(subfolder_ids)

# === Función: obtiene ROI desde el video (igual que tu pipeline) ===
def process_video_and_get_crop(video, min_obj_size=1000, hole_size=1000):
    video = video[:, :, 0:-50, :]  # elimina borde derecho (artefacto)
    D, H, W, _ = video.shape
    ref_idx, idx1, idx2 = random.sample(range(D), 3)
    diff1 = (video[ref_idx].astype(np.float32) - video[idx1].astype(np.float32)).astype(np.uint8)
    diff2 = (video[ref_idx].astype(np.float32) - video[idx2].astype(np.float32)).astype(np.uint8)
    gray1 = cv2.cvtColor(diff1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(diff2, cv2.COLOR_RGB2GRAY)
    mask = np.logical_or(gray1 > 0, gray2 > 0)
    mask = skm.binary_erosion(mask)
    mask = skm.remove_small_objects(mask, min_size=min_obj_size)
    mask = skm.remove_small_holes(mask, area_threshold=hole_size)
    mask = skm.opening(mask)
    labels = measure.label(mask)
    props = measure.regionprops(labels)
    if not props:
        raise RuntimeError("No se encontró ningún objeto tras la limpieza.")
    largest = max(props, key=lambda p: p.area)
    minr, minc, maxr, maxc = largest.bbox
    cropped_video = video[:, minr:maxr, minc:maxc, :]
    return cropped_video, (minr, maxr, minc, maxc)

# === Resize de label map con NN a 128³ ===
def resize_label_128(label_vol_DHW):
    # TorchIO espera (C, D, H, W); usamos C=1
    lbl = label_vol_DHW.astype(np.int16)  # o uint8/uint16 según tus clases
    img = tio.LabelMap(tensor=lbl[None, ...])  # (1, D, H, W)
    img = tio.Resize((128, 128, 128))(img)     # NN por ser LabelMap
    out = img.data.numpy()[0]                  # (D, H, W)
    return out

def resize_video_128(video_DHWC):
    # Conversión a escala de grises y resize a (128,128,128)
    _, H, W, _ = video_DHWC.shape
    # gray_vol = np.zeros((1, D, H, W), dtype=np.uint8)
    # for i in range(D):
    #     gray = np.dot(video[i], [0.2989, 0.5870, 0.1140])
    #     gray_vol[0, i] = gray.astype(np.uint8)
    # coefs float32 para no caer en float64
    coefs = np.array([0.2989, 0.5870, 0.1140], dtype=np.float32)
    # → esto te da float32
    gray = np.tensordot(video_DHWC, coefs, axes=([3], [0]))

    # normaliza en [0,1] y SIGUE siendo float32
    gray = gray / np.float32(255.0)

    # 3. Construye el ScalarImage YA en float32 normalizado
    img = tio.ScalarImage(tensor=gray[None, ...])  # shape (1,D,H,W), dtype float32
    #solo cambiar tamaño temporal a 128, manteniendo H W sin cambiar
    img = tio.Resize((128, H, W))(img)
    out = img.data.numpy()  # shape (1,128,128,128), dtype float32
    return out

def gt_to_128_from_video(gt_DHW, video_path, roi_coords=None, trim_right_px=50):
    """
    gt_DHW: volumen GT (D,H,W)
    video_path: mp4 correspondiente (mismo índice)
    roi_coords: (minr, maxr, minc, maxc) si ya lo tienes; si no, se calcula del video
    """
    crop_flag=False
    # 1) Cargar video y (si no hay coords) calcular ROI exactamente como en inferencia
    video = sio.vread(video_path)  # (D,H,W,3)
    if roi_coords is None:
        cropped_video, roi_coords = process_video_and_get_crop(video)
    minr, maxr, minc, maxc = roi_coords

    # 2) Aplicar recorte de borde derecho también al GT
    if trim_right_px and gt_DHW.shape[2] > trim_right_px:
        gt_DHW = gt_DHW[:, :, 0:-trim_right_px]

    # 3) Recortar GT con el mismo ROI del video
    gt_crop = gt_DHW[:, minr:maxr, minc:maxc]
    if gt_crop.size != cropped_video[:,:,:,0].size:
        crop_flag = True
        print(f"El recorte del GT {gt_crop.shape} no coincide con el del video {cropped_video.shape}")
        
    # 4) Resize NN a 128³
    gt_128 = resize_label_128(gt_crop)
    cropped_video = resize_video_128(cropped_video)[0]  # (128, H', W', 3)
    return gt_128, roi_coords,cropped_video,crop_flag




output_dir = "/data/GitHub/Breast-AI-model/src/metrics_rc2"
os.makedirs(output_dir, exist_ok=True)

metric_list = ["DSC", "IoU", "AHD", "VS", "Accuracy", "Sensitivity", "Specificity", "AUC","Kap"]

def safe_eval(gt, pred, metric):
    try:
        return evaluate(gt, pred, metric=metric)
    except Exception as e:
        # opcional: print(f"[WARN] {metric} falló: {e}")
        return np.nan

# Acumuladores
per_video_rows = []

subfolders_ids = natsort.natsorted(get_subfolder_ids('/data/GitHub/Breast-AI-model/src/predictions_rc2/'))
count_patients = 0

for subfolder_id in subfolders_ids:
    ground_truth_folder = os.path.join(ground_truth_path, subfolder_id, 'FINAL LABELS')
    if not os.path.exists(ground_truth_folder):
        ground_truth_folder = os.path.join(ground_truth_path, subfolder_id, 'Segmentations')
        if not os.path.exists(ground_truth_folder):
            continue
    count_patients += 1

    ground_truth_files = glob.glob(os.path.join(ground_truth_folder, '**', '*.mat'), recursive=True)
    ground_truth_files = natsort.natsorted([f for f in ground_truth_files
                                            if 'groundTruthMed.mat' not in f and 'Session.mat' not in f])

    pred_files = glob.glob(os.path.join('/data/GitHub/Breast-AI-model/src/predictions_rc2', subfolder_id, '*.npy'))
    pred_files = natsort.natsorted(pred_files)

    patterns = [
    os.path.join(videos_path, subfolder_id, 'VSI 8 [cC]lips', '*.[mM][pP]4'),
    os.path.join(videos_path, subfolder_id, '8 VSI [cC]lips', '*.[mM][pP]4'),  # opcional, por si existe esa variante
]

    videos_files = []
    for pat in patterns:
        videos_files += glob.glob(pat)

    # quitar duplicados y ordenar de forma natural
    videos_files = natsort.natsorted({os.path.realpath(p) for p in videos_files})
    # idx -> path
    npy_by_idx = {get_npy_idx(p): p for p in pred_files if get_npy_idx(p) is not None}
    videos_by_idx = {get_video_idx(v): v for v in videos_files if get_video_idx(v) is not None}

    # (gt_mat, pred_npy, video_mp4, idx)
    pairs = []
    for mp in ground_truth_files:
        i = get_mat_idx(mp)
        if i is None:
            continue
        pred_p = npy_by_idx.get(i)
        vid_p  = videos_by_idx.get(i)
        if pred_p is not None and vid_p is not None:   # si no hay video, omitimos porque el ROI depende del video
            pairs.append((mp, pred_p, vid_p, i))

    print(f"[{subfolder_id}] {len(pairs)} pares válidos")

    # ---- Métricas por video ----
    for gt_file, pred_file, vid_file, idx in pairs:
        try:
            # carga GT, alinea y redimensiona
            gt = np.transpose(loadmat(gt_file)['labels'], (2, 0, 1))  # (D,H,W)
            gt_128, coords, cropped_video,crop_flag = gt_to_128_from_video(gt, vid_file)

            # carga pred
            pred = np.load(pred_file)

            # binariza por seguridad
            gt_bin   = (gt_128 > 0).astype(np.uint8)
            pred_bin = (pred   > 0).astype(np.uint8)

            # evalúa todas las métricas (con tolerancia a fallos)
            row = {
                "subject_id": subfolder_id.split(' ')[-1],
                "sweep": idx,
                "D": gt_bin.shape[0], "H": gt_bin.shape[1], "W": gt_bin.shape[2],
                'Crop flag': crop_flag
            }
            for m in metric_list:
                row[m] = np.round(safe_eval(gt_bin, pred_bin, metric=m),2)

            per_video_rows.append(row)

        except Exception as e:
            # Loguea y sigue con el siguiente video
            print(f"[WARN] Falló subject={subfolder_id} idx={idx}: {e}")
            continue
# ---- DataFrames y CSVs ----
df_videos = pd.DataFrame(per_video_rows)

# CSV 1: métricas individuales por video
csv_per_video = os.path.join(output_dir, "metrics_per_video.csv")
df_videos.to_csv(csv_per_video, index=False)
print(f"✅ Guardado: {csv_per_video}")

# === NORMALIZAR IDs EN df_videos (crea un ID canónico numérico) ===
# subject_id típicamente es "Patient 2", "Patient 03", etc. Extraemos el entero.
df_videos['patient_num'] = (
    df_videos['subject_id'].astype(str).str.extract(r'(\d+)')[0].astype('Int64')
)

# CSV 2: promedios por paciente (ignora NaN) + Crop flag
agg_cols = metric_list  # métricas a promediar
agg_dict = {m: 'mean' for m in agg_cols}
agg_dict['Crop flag'] = 'max'   # True si algún video del paciente tuvo Crop flag

df_patients = (
    df_videos
    .groupby(['subject_id', 'patient_num'], dropna=False)
    .agg(agg_dict)
    .reset_index()
)

# ==== MERGE CON BS.csv (usa patient_num) ====
bs_csv = '/data/GitHub/Breast-AI-model/src/BS.csv'  # ajusta si es necesario
bs = pd.read_csv(bs_csv)

# Normaliza nombres y tipos
bs.columns = [c.strip() for c in bs.columns]
esperadas = ['ID', 'Mass/no mass', 'Cancer/no cancer', 'Category']
faltantes = [c for c in esperadas if c not in bs.columns]
if faltantes:
    raise ValueError(f"Faltan columnas en BS.csv: {faltantes}")

# Crea también el ID canónico en BS.csv
bs['ID_str'] = bs['ID'].astype(str).str.strip()
bs['patient_num'] = bs['ID_str'].str.extract(r'(\d+)')[0].astype('Int64')
bs = bs.drop_duplicates(subset='patient_num', keep='first')

# Merge por patient_num
df_patients_labeled = (
    df_patients.merge(
        bs[['patient_num','Mass/no mass','Cancer/no cancer','Category']],
        on='patient_num', how='left'
    )
)

# (Opcional) mover columnas de etiqueta al inicio
label_cols = ['Mass/no mass','Cancer/no cancer','Category']
ordered_cols = (['subject_id','patient_num','Crop flag'] + label_cols +
                [c for c in df_patients_labeled.columns if c not in (['subject_id','patient_num','Crop flag'] + label_cols)])
df_patients_labeled = df_patients_labeled[ordered_cols]

# Reporta pacientes sin match
sin_match = df_patients_labeled[df_patients_labeled['Mass/no mass'].isna()]['subject_id'].tolist()
if sin_match:
    print(f"[INFO] {len(sin_match)} pacientes sin match en BS.csv (ejemplos): {sin_match[:10]}")


csv_per_patient_labeled = os.path.join(output_dir, "metrics_per_patient_mean.csv")
df_patients_labeled.set_index('subject_id').drop(columns=['patient_num']).to_csv(csv_per_patient_labeled)
print(f"✅ Guardado con etiquetas: {csv_per_patient_labeled}")

# CSV 3: promedio global del dataset (ignora NaN)
global_means = df_videos[agg_cols].mean(numeric_only=True)
df_global = pd.DataFrame(global_means).T
df_global.index = ["dataset_mean"]
csv_global = os.path.join(output_dir, "metrics_dataset_mean.csv")
df_global.to_csv(csv_global)
print(f"✅ Guardado: {csv_global}")


        #Mostrar el primer frame no vacío del gt_128
        #first_non_empty_frame = np.argmax(np.any(gt_128, axis=(1, 2)))
        #mostrar el video, el ground_truth y la predicción
        # if first_non_empty_frame < gt_128.shape[0]:
        #     plt.figure(figsize=(12, 4))
        #     plt.subplot(1, 3, 1)
        #     plt.imshow(gt_128[first_non_empty_frame], cmap='gray')
        #     plt.title('Ground Truth Frame')
        #     plt.axis('off')

        #     plt.subplot(1, 3, 2)
        #     plt.imshow(pred[first_non_empty_frame], cmap='gray')
        #     plt.title('Prediction Frame')
        #     plt.axis('off')

        #     plt.subplot(1, 3, 3)
        #     plt.imshow(cropped_video[first_non_empty_frame], cmap='gray')
        #     plt.title  # 2) Aplicar recorte de borde derecho también al GT
        #     ('Video Frame')
        #     plt.axis('off')

        #     plt.show()
        #print("Shapes -> GT:", ground_truth.shape, "| Pred:", prediction.shape, "| Video:", video.shape)
  

    #break
print(f"Total patients processed: {count_patients}")
print(f"Total raw patients: {len(subfolders_ids)}")


[Patient 2] 4 pares válidos
[Patient 3] 4 pares válidos
El recorte del GT (393, 1047, 317) no coincide con el del video (393, 1075, 865, 3)
El recorte del GT (364, 1049, 318) no coincide con el del video (364, 1077, 866, 3)
El recorte del GT (244, 1049, 317) no coincide con el del video (244, 1070, 860, 3)
El recorte del GT (305, 1048, 317) no coincide con el del video (305, 1076, 865, 3)
[Patient 9] 4 pares válidos
El recorte del GT (291, 1059, 316) no coincide con el del video (291, 1075, 859, 3)
El recorte del GT (331, 1059, 317) no coincide con el del video (331, 1075, 866, 3)
El recorte del GT (297, 1058, 317) no coincide con el del video (297, 1074, 866, 3)
El recorte del GT (289, 1060, 317) no coincide con el del video (289, 1074, 860, 3)
[Patient 12] 4 pares válidos
[Patient 13] 4 pares válidos
El recorte del GT (301, 1052, 318) no coincide con el del video (301, 1075, 861, 3)
El recorte del GT (201, 1017, 318) no coincide con el del video (201, 1017, 860, 3)
El recorte del GT 

# Generacion de OVERLAYS

In [61]:
import glob
import os
import natsort
import numpy as np
from torchmetrics import Dice, JaccardIndex, Precision, Recall, Specificity, Accuracy
import re
from scipy.io import loadmat
import skvideo.io as sio
import random
import torchio as tio
import skimage.morphology as skm
from skimage import measure
import cv2
import matplotlib.pyplot as plt
import torch
from miseval import evaluate
import pandas as pd

from skimage.segmentation import find_boundaries



def draw_label(img_rgb, text, org, font_scale=0.8, color=(255,255,255)):
    cv2.putText(img_rgb, text, org, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0,0,0), 4, cv2.LINE_AA)
    cv2.putText(img_rgb, text, org, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, 2, cv2.LINE_AA)

def overlay_mask_on_frame(frame_rgb, mask_2d, color=(0,255,0), alpha=0.35, draw_contour=True):
    out = frame_rgb.copy()
    m = mask_2d.astype(bool)
    if m.any():
        color_arr = np.array(color, dtype=np.uint8)
        out[m] = (out[m] * (1 - alpha) + color_arr * alpha).astype(np.uint8)
        if draw_contour:
            edges = find_boundaries(mask_2d, mode="inner")
            out[edges] = color_arr
    return out

def make_even_frame(img_rgb):
    """Asegura H y W pares para yuv420p con padding de 1 px si hace falta."""
    h, w = img_rgb.shape[:2]
    pad_h = h % 2
    pad_w = w % 2
    if pad_h or pad_w:
        img_rgb = np.pad(img_rgb, ((0, pad_h), (0, pad_w), (0, 0)), mode='edge')
    # garantizar memoria contigua y dtype correcto
    return np.ascontiguousarray(img_rgb.astype(np.uint8))

def write_triptych_skvideo(orig_DHWC, gt_DHW, pred_DHW, out_path, fps=25.0, method="ffmpegwriter"):
    D, H, W, _ = orig_DHWC.shape
    assert gt_DHW.shape == (D, H, W) and pred_DHW.shape == (D, H, W)

    if method == "vwrite":
        frames = []
        for t in range(D):
            f0 = orig_DHWC[t]
            f1 = overlay_mask_on_frame(f0, gt_DHW[t],  color=(0,255,0))
            f2 = overlay_mask_on_frame(f0, pred_DHW[t], color=(255,0,255))
            trip = np.concatenate([f0, f1, f2], axis=1)
            draw_label(trip, "Original", (15, 34))
            draw_label(trip, "GT",       (15 + W, 34))
            draw_label(trip, "Pred",     (15 + 2*W, 34))
            trip = make_even_frame(trip)
            frames.append(trip)
        vid = np.stack(frames, axis=0)
        sio.vwrite(out_path, vid, outputdict={'-r': str(fps), '-vcodec': 'libx264', '-pix_fmt': 'yuv420p'})
    else:
        writer = sio.FFmpegWriter(out_path, outputdict={'-r': str(fps), '-vcodec': 'libx264', '-pix_fmt': 'yuv420p'})
        for t in range(D):
            f0 = orig_DHWC[t]
            f1 = overlay_mask_on_frame(f0, gt_DHW[t],  color=(0,255,0))
            f2 = overlay_mask_on_frame(f0, pred_DHW[t], color=(255,0,255))
            trip = np.concatenate([f0, f1, f2], axis=1)
            draw_label(trip, "Original", (15, 34))
            draw_label(trip, "GT",       (15 + W, 34))
            draw_label(trip, "Pred",     (15 + 2*W, 34))
            trip = make_even_frame(trip)
            writer.writeFrame(trip)
        writer.close()
    print(f"🎥 guardado: {out_path}")

# === Resize de label map con NN a 128³ ===
def resize_pred_n(label_vol_DHW,D, H, W):
    # TorchIO espera (C, D, H, W); usamos C=1
    lbl = label_vol_DHW.astype(np.int8)  # o uint8/uint16 según tus clases
    img = tio.LabelMap(tensor=lbl[None, ...])  # (1, D, H, W)
    img = tio.Resize((D,H,W))(img)     # NN por ser LabelMap
    out = img.data.numpy()[0]                  # (D, H, W)
    return out


output_dir = "/data/GitHub/Breast-AI-model/src/overlay_test_rc2"
os.makedirs(output_dir, exist_ok=True)

def safe_eval(gt, pred, metric):
    try:
        return evaluate(gt, pred, metric=metric)
    except Exception as e:
        # opcional: print(f"[WARN] {metric} falló: {e}")
        return np.nan

# Acumuladores
per_video_rows = []

subfolders_ids = natsort.natsorted(get_subfolder_ids('/data/GitHub/Breast-AI-model/src/predictions_rc2/'))
count_patients = 0

for subfolder_id in subfolders_ids:
    ground_truth_folder = os.path.join(ground_truth_path, subfolder_id, 'FINAL LABELS')
    if not os.path.exists(ground_truth_folder):
        ground_truth_folder = os.path.join(ground_truth_path, subfolder_id, 'Segmentations')
        if not os.path.exists(ground_truth_folder):
            continue
    count_patients += 1

    ground_truth_files = glob.glob(os.path.join(ground_truth_folder, '**', '*.mat'), recursive=True)
    ground_truth_files = natsort.natsorted([f for f in ground_truth_files
                                            if 'groundTruthMed.mat' not in f and 'Session.mat' not in f])

    pred_files = glob.glob(os.path.join('/data/GitHub/Breast-AI-model/src/predictions_rc2', subfolder_id, '*.npy'))
    pred_files = natsort.natsorted(pred_files)

    patterns = [
    os.path.join(videos_path, subfolder_id, 'VSI 8 [cC]lips', '*.[mM][pP]4'),
    os.path.join(videos_path, subfolder_id, '8 VSI [cC]lips', '*.[mM][pP]4'),  # opcional, por si existe esa variante
]

    videos_files = []
    for pat in patterns:
        videos_files += glob.glob(pat)

    # quitar duplicados y ordenar de forma natural
    videos_files = natsort.natsorted({os.path.realpath(p) for p in videos_files})
    # idx -> path
    npy_by_idx = {get_npy_idx(p): p for p in pred_files if get_npy_idx(p) is not None}
    videos_by_idx = {get_video_idx(v): v for v in videos_files if get_video_idx(v) is not None}

    # (gt_mat, pred_npy, video_mp4, idx)
    pairs = []
    for mp in ground_truth_files:
        i = get_mat_idx(mp)
        if i is None:
            continue
        pred_p = npy_by_idx.get(i)
        vid_p  = videos_by_idx.get(i)
        if pred_p is not None and vid_p is not None:   # si no hay video, omitimos porque el ROI depende del video
            pairs.append((mp, pred_p, vid_p, i))

    print(f"[{subfolder_id}] {len(pairs)} pares válidos")

    # ---- Métricas por video ----
    # Por cada par
    for gt_file, pred_file, vid_file, idx in pairs:
        try:
            video = sio.vread(vid_file)  # (D,H,W,3)
            
            cropped_video, roi_coords = process_video_and_get_crop(video)      # (D',H',W',3)
            gt_DHW = np.transpose(loadmat(gt_file)['labels'], (2,0,1))      # (D,H,W)
            minr, maxr, minc, maxc = roi_coords
            trim_right_px=50

            # 2) Aplicar recorte de borde derecho también al GT
            if trim_right_px and gt_DHW.shape[2] > trim_right_px:
                gt_DHW = gt_DHW[:, :, 0:-trim_right_px]

            # 3) Recortar GT con el mismo ROI del video
            gt_bin = gt_DHW[:, minr:maxr, minc:maxc]

            print(video.shape, cropped_video.shape, gt_DHW.shape, gt_bin.shape)


            pred = np.load(pred_file)                                   # (128,128,128)
            pred_bin = (pred > 0).astype(np.uint8)
            pred_rs  = resize_pred_n(pred_bin, gt_bin.shape[0], gt_bin.shape[1], gt_bin.shape[2])   # (D',H',W')

            fps = get_fps(vid_file, fallback=25.0)
            out_name = f"{subfolder_id}_{idx:2d}_overlay.mp4"
            out_path = os.path.join(output_dir, out_name)

            # Usa vwrite si lo quieres explícito: method="vwrite"
            write_triptych_skvideo(cropped_video, gt_bin, pred_rs, out_path, fps=fps, method="ffmpegwriter")

        except Exception as e:
            print(f"[WARN] Overlay fallo subject={subfolder_id} idx={idx}: {e}")
            continue


print(f"Total patients processed: {count_patients}")
print(f"Total raw patients: {len(subfolders_ids)}")


[Patient 2] 4 pares válidos
(273, 1080, 1854, 3) (273, 1076, 866, 3) (273, 1080, 1804) (273, 1076, 866)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 2_ 1_overlay.mp4
(361, 1080, 1854, 3) (361, 1075, 866, 3) (361, 1080, 1804) (361, 1075, 866)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 2_ 4_overlay.mp4
(301, 1080, 1854, 3) (301, 1077, 860, 3) (301, 1080, 1804) (301, 1077, 860)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 2_ 5_overlay.mp4
(332, 1080, 1854, 3) (332, 1075, 862, 3) (332, 1080, 1804) (332, 1075, 862)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 2_ 8_overlay.mp4
[Patient 3] 4 pares válidos
(393, 1080, 1854, 3) (393, 1077, 861, 3) (393, 1052, 811) (393, 1049, 318)
[WARN] Overlay fallo subject=Patient 3 idx=2: 
(364, 1080, 1854, 3) (364, 1064, 859, 3) (364, 1052, 811) (364, 1036, 317)
[WARN] Overlay fallo subject=Patient 3 idx=4: 
(244, 1080, 1854, 3) (244, 1073, 862, 3) (244, 1

/home/exx/anaconda3/envs/ligthning/lib/python3.12/site-packages/torchio/transforms/transform.py:162: RuntimeWarning: Output shape (265, 1075, 858) != target shape (265, 1075, 857). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 12_ 6_overlay.mp4
(239, 1080, 1854, 3) (239, 842, 859, 3) (239, 1080, 1804) (239, 842, 859)


/home/exx/anaconda3/envs/ligthning/lib/python3.12/site-packages/torchio/transforms/transform.py:162: RuntimeWarning: Output shape (240, 842, 859) != target shape (239, 842, 859). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 12_ 8_overlay.mp4
[Patient 13] 4 pares válidos
(301, 1080, 1854, 3) (301, 1067, 862, 3) (301, 1057, 812) (301, 1044, 319)
[WARN] Overlay fallo subject=Patient 13 idx=1: 
(201, 1080, 1854, 3) (201, 1033, 861, 3) (201, 1057, 812) (201, 1033, 319)
[WARN] Overlay fallo subject=Patient 13 idx=4: 
(185, 1080, 1854, 3) (185, 1075, 860, 3) (185, 1057, 812) (185, 1054, 318)
[WARN] Overlay fallo subject=Patient 13 idx=6: 
(271, 1080, 1854, 3) (271, 1075, 860, 3) (271, 1057, 812) (271, 1054, 318)
[WARN] Overlay fallo subject=Patient 13 idx=8: 
[Patient 18] 4 pares válidos
(249, 1080, 1854, 3) (249, 1064, 832, 3) (249, 1080, 1804) (249, 1064, 832)


...

🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 113_ 2_overlay.mp4
(293, 1080, 858, 3) (293, 855, 692, 3) (293, 1080, 808) (293, 855, 692)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 113_ 4_overlay.mp4
(235, 1080, 858, 3) (235, 856, 689, 3) (235, 1080, 808) (235, 856, 689)


/home/exx/anaconda3/envs/ligthning/lib/python3.12/site-packages/torchio/transforms/transform.py:162: RuntimeWarning: Output shape (235, 857, 689) != target shape (235, 856, 689). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 113_ 6_overlay.mp4
(285, 1080, 858, 3) (285, 851, 691, 3) (285, 1080, 808) (285, 851, 691)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 113_ 8_overlay.mp4
[Patient 115a] 4 pares válidos
(259, 1080, 858, 3) (259, 851, 692, 3) (259, 1080, 808) (259, 851, 692)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 115a_ 2_overlay.mp4
(209, 1080, 858, 3) (209, 855, 764, 3) (209, 1080, 808) (209, 855, 764)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 115a_ 3_overlay.mp4
(187, 1080, 858, 3) (187, 855, 729, 3) (187, 1080, 808) (187, 855, 729)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 115a_ 5_overlay.mp4
(241, 1080, 858, 3) (241, 853, 693, 3) (241, 1080, 808) (241, 853, 693)


/home/exx/anaconda3/envs/ligthning/lib/python3.12/site-packages/torchio/transforms/transform.py:162: RuntimeWarning: Output shape (241, 854, 693) != target shape (241, 853, 693). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 115a_ 8_overlay.mp4
[Patient 117] 4 pares válidos
(446, 1080, 858, 3) (446, 855, 694, 3) (446, 1080, 808) (446, 855, 694)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 117_ 2_overlay.mp4
(504, 1080, 858, 3) (504, 859, 691, 3) (504, 1080, 808) (504, 859, 691)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 117_ 4_overlay.mp4
(614, 1080, 858, 3) (614, 855, 761, 3) (614, 1080, 808) (614, 855, 761)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 117_ 5_overlay.mp4
(526, 1080, 858, 3) (526, 855, 808, 3) (526, 1080, 808) (526, 855, 808)
🎥 guardado: /data/GitHub/Breast-AI-model/src/overlay_test_rc2/Patient 117_ 8_overlay.mp4
Total patients processed: 65
Total raw patients: 107
